In [1]:
#Mojo of reproducibility
import torch
import random
import numpy as np

def set_seed(seed):
  #PyTorch
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  #Numpy
  np.random.seed(seed)
  #Python_random
  random.seed(seed)
  #CuDNN (when using CUDA)
  if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/takehararyoutarou/anaconda3/envs/mlVenv/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/takehararyoutarou/anaconda3/envs/mlVenv/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/takehararyoutarou/anaconda3/envs/mlVenv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/takehararyoutarou/anaconda3/envs/mlVenv/lib/python3.10/site-pa

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class baseLineClassifier(nn.Module):
    def __init__(self, hidden_size, num_layers, dropout_rate):
        super().__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Define a list of layers
        self.layers = nn.ModuleList()
        
        # Define the first layer
        self.layers.append(nn.Linear(173, hidden_size))
        
        # Define the intermediate hidden layers
        for _ in range(num_layers - 2):
            self.layers.append(nn.Linear(hidden_size, hidden_size))
        
        # Final layer to output
        self.emo_output_layer = nn.Linear(hidden_size, 6)
        self.strength_output_layer = nn.Linear(hidden_size, 3)

        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)

        # Initialize weights
        self.init_weights()

    def init_weights(self):
        for layer in self.modules():
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias)

    def forward(self,x):
        # Pass through hidden layers
        for layer in self.layers:
            x = F.relu(layer(x))
            x = self.dropout(x)

        # Output layers
        emo_output = self.emo_output_layer(x)
        strength_output = self.strength_output_layer(x)
        
        return emo_output, strength_output

In [3]:
# Create dataset
from torch.utils.data import Dataset

class voiceDataset(Dataset):
    def __init__(self, features, emotionLabels, strengthLabels):
        self.features = features
        self.emotionLabels = emotionLabels
        self.strengthLabels = strengthLabels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return {'features':self.features[idx], 
                'emotionLabel':self.emotionLabels[idx], 
                'strengthLabel':self.strengthLabels[idx]
        }   


In [4]:
import pandas as pd
df = pd.read_csv('featuresAndLabels.csv')

In [5]:
# Cut corresponding columns of df into features and labels
# Turn them into tensors
features = df.iloc[:, 1:174].values
features = torch.tensor(features, dtype=torch.float32)

emotionLabels = df.iloc[:,174:180].values
emotionLabels = torch.tensor(emotionLabels, dtype=torch.float32)

strengthLabels = df.iloc[:,180:183].values
strengthLabels = torch.tensor(strengthLabels, dtype=torch.float32)

In [6]:
# Create datasets
# train, validate, test = 8:1:1
train_size = int(0.8 * len(df))
val_size = int(0.1 * len(df))
test_size = len(df) - train_size - val_size

print(train_size, val_size, test_size)

train_dataset = voiceDataset(features[:train_size], 
                             emotionLabels[:train_size], 
                             strengthLabels[:train_size])

validate_dataset = voiceDataset(features[train_size:train_size + val_size],
                                emotionLabels[train_size:train_size + val_size], 
                                strengthLabels[train_size:train_size + val_size])

test_dataset = voiceDataset(features[train_size + val_size:],
                            emotionLabels[train_size + val_size:], 
                            strengthLabels[train_size + val_size:])


from torch.utils.data import DataLoader

# Create dataLoader
trainDataLoader = DataLoader(train_dataset, batch_size=64)
validationDataLoader = DataLoader(validate_dataset,batch_size=64)
testDataLoader = DataLoader(test_dataset,batch_size=1)


5953 744 745


In [7]:
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Define the trainer & validator function
def train_validate_model(hidden_size, num_layers,dropout_rate, trainDataLoader, 
                         validationDataLoader,num_epochs, learning_rate,
                         fineTuning= False):

    set_seed(42)

    #Get the GPU as a device if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    #Instantiate the model
    model = baseLineClassifier(hidden_size = hidden_size,
                           num_layers = num_layers,
                           dropout_rate = dropout_rate)

    # Moving the model to GPU if available
    model.to(device)

    #Prepare the optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    #Prepare the error function
    criterion = nn.CrossEntropyLoss()

    #Prepare the scheduler
    #Reduce the learning rate by 0.1 if the validation loss does not decrease for 3 epochs
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

    #Prepare the DataLoader
    train_data_loader = trainDataLoader
    validation_data_loader = validationDataLoader

    #Placeholder for minimum validation loss
    min_val_loss = float('inf')

    for epoch in range(num_epochs):
        #set the model to training mode
        model.train()

        total_loss = 0 #Placeholder for training loss per epoch
        for batch in train_data_loader:
            feature = batch['features'].to(device)
            emotionLabel = batch['emotionLabel'].to(device)
            strengthLabel = batch['strengthLabel'].to(device)

            optimizer.zero_grad()
            emotionOutput, strengthOutput = model(feature)
            #Calculate the loss for emotion head
            emo_loss = criterion(emotionOutput, emotionLabel)
            #Calculate the loss for strength head
            strength_loss = criterion(strengthOutput, strengthLabel)
            #Combine two losses to make a total loss. 
            #Put more weight on the emotion loss (7:3). Detecting emotion is more critical
            loss = 0.7*emo_loss + 0.3*strength_loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        #Validate the model
        model.eval() #Set the model to evaluation mode
        with torch.no_grad():
            total_samples = 0 #Total output for accuracy calculation
            emoCorrect = 0 #Amount of correct prediction for accuracy calculation
            strengthCorrect = 0
            total_val_loss = 0 #Placeholder for validation loss per epoch
            
            for batch in validation_data_loader:
                feature = batch['features'].to(device)
                emotionLabel = batch['emotionLabel'].to(device)
                strengthLabel = batch['strengthLabel'].to(device)
                #Forward pass
                emotionOutput, strengthOutput = model(feature)
                emo_loss = criterion(emotionOutput, emotionLabel)
                strength_loss = criterion(strengthOutput, strengthLabel )
                total_val_loss += 0.7*emo_loss + 0.3*strength_loss
                
                # Get predicted emotion class & target emotion class
                emo_predicted = torch.argmax(emotionOutput, dim=1)
                emo_target = torch.argmax(emotionLabel, dim=1)
                
                # Get predicted strength class & target emotion class
                strength_predicted = torch.argmax(strengthOutput,dim=1)
                strength_target = torch.argmax(strengthLabel, dim=1 )
                
                emoCorrect += (emo_predicted == emo_target).sum().item()
                strengthCorrect += (strength_predicted ==strength_target).sum().item()
                
                #Get total number of samples per epoch
                total_samples += emotionLabel.size(0)

            emo_accuracy = emoCorrect / total_samples
            strength_accuracy = strengthCorrect / total_samples


            #Print out the validation loss and accuracy per epoch
            print(f"Epoch {epoch+1}/{num_epochs} | Training Loss: {total_loss:.4f}, Validation Loss: {total_val_loss:.4f}, Accuracy (Emotion): {emo_accuracy:.4f}, Accuracy (Strength): {strength_accuracy}")

            #pass the validation loss to the scheduler
            scheduler.step(total_val_loss)

        #If fineTuning = False, save the model with the lowest validation loss
        #Save the first epoch model just in case

            if epoch == 0:
                min_val_loss = total_val_loss #Instantiate the min_val_loss at the first epoch
                torch.save(model.state_dict(), 'bestBaseLine.pth')
            #Save the model if the validation loss is the lowest
            elif total_val_loss < min_val_loss:
                min_val_loss = total_val_loss
                torch.save(model.state_dict(), 'bestBaseLine.pth')
                print(f"Model saved after Epoch: {epoch+1}")

    #Return the minimum validation loss for hyperparameter tuning
    return min_val_loss

In [8]:
import optuna

def objective(trial):

    set_seed(42)

    #Define the hyperparameters to be tuned
    #Dimension of the hidden layer 
    hidden_size = trial.suggest_int('hidden_size', 50, 100)
    #Number of layers
    num_layers = trial.suggest_int('num_layers', 5, 10)
    #Dropout rate for the final feedforward network [0.1, 0.5]
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    #Number of epochs [10, 30]
    num_epochs = trial.suggest_int('num_epochs', 10, 30)
    #Learning rate for the optimizer [1e-5, 1e-3]
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3)

    result = train_validate_model(hidden_size = hidden_size,
                                  num_layers = num_layers,
                                  dropout_rate = dropout_rate,
                                  trainDataLoader = trainDataLoader,
                                  validationDataLoader = validationDataLoader,
                                  num_epochs = num_epochs,
                                  learning_rate = learning_rate
                                  )

    return result


/Users/takehararyoutarou/anaconda3/envs/mlVenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Print the best hyperparameters
print("Best trial:")
trial = study.best_trial
print(f"  Value (Validation Loss): {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-04-06 21:32:40,975] A new study created in memory with name: no-name-2fe2c2a9-4b7f-4144-b309-8c41b4021a6f
/Users/takehararyoutarou/anaconda3/envs/mlVenv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/23 | Training Loss: 646.4708, Validation Loss: 18.3937, Accuracy (Emotion): 0.3414, Accuracy (Strength): 0.5577956989247311
Epoch 2/23 | Training Loss: 173.6385, Validation Loss: 18.2262, Accuracy (Emotion): 0.5833, Accuracy (Strength): 0.8010752688172043
Model saved after Epoch: 2
Epoch 3/23 | Training Loss: 148.8969, Validation Loss: 17.9308, Accuracy (Emotion): 0.6102, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 3
Epoch 4/23 | Training Loss: 141.0022, Validation Loss: 17.6239, Accuracy (Emotion): 0.6102, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 4
Epoch 5/23 | Training Loss: 138.0353, Validation Loss: 17.3678, Accuracy (Emotion): 0.6102, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 5
Epoch 6/23 | Training Loss: 136.0867, Validation Loss: 17.0774, Accuracy (Emotion): 0.6102, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 6
Epoch 7/23 | Training Loss: 134.1281, Validation Loss: 16.8699, Accuracy 

[I 2025-04-06 21:32:44,630] Trial 0 finished with value: 16.202356338500977 and parameters: {'hidden_size': 51, 'num_layers': 7, 'dropout_rate': 0.2913058907912604, 'num_epochs': 23, 'learning_rate': 0.0006057716449734846}. Best is trial 0 with value: 16.202356338500977.


Epoch 23/23 | Training Loss: 128.3466, Validation Loss: 16.2024, Accuracy (Emotion): 0.6102, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 23
Epoch 1/20 | Training Loss: 796.6925, Validation Loss: 18.0718, Accuracy (Emotion): 0.3938, Accuracy (Strength): 0.41263440860215056
Epoch 2/20 | Training Loss: 191.4046, Validation Loss: 17.8569, Accuracy (Emotion): 0.5672, Accuracy (Strength): 0.782258064516129
Model saved after Epoch: 2
Epoch 3/20 | Training Loss: 153.1341, Validation Loss: 17.7009, Accuracy (Emotion): 0.6129, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 3
Epoch 4/20 | Training Loss: 143.0786, Validation Loss: 17.5108, Accuracy (Emotion): 0.6116, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 4
Epoch 5/20 | Training Loss: 138.5905, Validation Loss: 17.3305, Accuracy (Emotion): 0.6102, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 5
Epoch 6/20 | Training Loss: 135.6444, Validation Loss: 17.2050, Accurac

[I 2025-04-06 21:32:47,030] Trial 1 finished with value: 15.932238578796387 and parameters: {'hidden_size': 65, 'num_layers': 6, 'dropout_rate': 0.27286428397042506, 'num_epochs': 20, 'learning_rate': 0.0006303539346617239}. Best is trial 1 with value: 15.932238578796387.


...

Epoch 17/18 | Training Loss: 123.5527, Validation Loss: 15.5050, Accuracy (Emotion): 0.6116, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 17
Epoch 18/18 | Training Loss: 122.9695, Validation Loss: 15.4500, Accuracy (Emotion): 0.6492, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 18
Epoch 1/21 | Training Loss: 247.0711, Validation Loss: 17.4396, Accuracy (Emotion): 0.6116, Accuracy (Strength): 0.8118279569892473
Epoch 2/21 | Training Loss: 138.6257, Validation Loss: 17.0513, Accuracy (Emotion): 0.6102, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 2
Epoch 3/21 | Training Loss: 133.9097, Validation Loss: 16.8771, Accuracy (Emotion): 0.6102, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 3
Epoch 4/21 | Training Loss: 132.0141, Validation Loss: 16.7172, Accuracy (Emotion): 0.6102, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 4
Epoch 5/21 | Training Loss: 130.8302, Validation Loss: 16.6953, Accur

[I 2025-04-06 21:35:00,001] Trial 48 finished with value: 15.382850646972656 and parameters: {'hidden_size': 68, 'num_layers': 9, 'dropout_rate': 0.11404826994723442, 'num_epochs': 21, 'learning_rate': 0.00039042593267715276}. Best is trial 23 with value: 14.973603248596191.


Epoch 21/21 | Training Loss: 121.9877, Validation Loss: 15.3829, Accuracy (Emotion): 0.6559, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 21
Epoch 1/13 | Training Loss: 357.8957, Validation Loss: 18.4190, Accuracy (Emotion): 0.6116, Accuracy (Strength): 0.7997311827956989
Epoch 2/13 | Training Loss: 146.7562, Validation Loss: 18.0100, Accuracy (Emotion): 0.6102, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 2
Epoch 3/13 | Training Loss: 138.9149, Validation Loss: 17.4636, Accuracy (Emotion): 0.6102, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 3
Epoch 4/13 | Training Loss: 134.8447, Validation Loss: 16.9478, Accuracy (Emotion): 0.6102, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 4
Epoch 5/13 | Training Loss: 132.7227, Validation Loss: 16.6017, Accuracy (Emotion): 0.6102, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 5
Epoch 6/13 | Training Loss: 131.2487, Validation Loss: 16.3933, Accurac

[I 2025-04-06 21:35:01,744] Trial 49 finished with value: 16.192359924316406 and parameters: {'hidden_size': 57, 'num_layers': 8, 'dropout_rate': 0.3163393507305805, 'num_epochs': 13, 'learning_rate': 0.0008497943257442877}. Best is trial 23 with value: 14.973603248596191.


Epoch 12/13 | Training Loss: 128.6578, Validation Loss: 16.1942, Accuracy (Emotion): 0.6102, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 12
Epoch 13/13 | Training Loss: 128.3993, Validation Loss: 16.1924, Accuracy (Emotion): 0.6102, Accuracy (Strength): 0.8145161290322581
Model saved after Epoch: 13
Best trial:
  Value (Validation Loss): 14.973603248596191
  Params: 
    hidden_size: 63
    num_layers: 9
    dropout_rate: 0.10326922793453225
    num_epochs: 20
    learning_rate: 0.0009911930030429713


In [10]:
set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = baseLineClassifier(
                       hidden_size = 16,
                       num_layers = 2,
                       dropout_rate = 0.2757662416167118,
                       output_size = 3
                       )

model.load_state_dict(torch.load('meme_model_lowestVL.pth',map_location=device))

model.eval() #Set the model to evaluation mode
model.to(device)

with torch.no_grad():
  total = 0 #Total output for accuracy calculation
  emoCorrect = 0 #Amount of correct prediction for accuracy calculation
  for batch in testDataLoader:
      feature = batch['feature'].to(device)
      emoLabel = batch['emotionLabel'].to(device)
      strengthLabel = batch['strengthLabel'].to(device)
      
      emoOutput, strengthOutput = model(feature)
      
      emoPredicted = torch.argmax(emoOutput, dim = 1)
      emoTarget = torch.argmax(emoLabel, dim=1)
      
      strengthPredicted = torch.argmax(strengthOutput, dim=1)
      strengthTarget = torch.argmax(strengthLabel, dim=1)
      
            
      total += emoLabel.size(0)
      emoCorrect += (emoPredicted == emoTarget).sum().item()
      strengthCorrect += (strengthPredicted == strengthTarget).sum().item()
      
  print(f"Emotion Accuracy: {emoCorrect/total:.4f}")
  print(f"Strength Accuracy: {strengthCorrect/total:.4f})

SyntaxError: unterminated string literal (detected at line 39) (2033693058.py, line 39)